In [1]:
import pandas as pd
import numpy as np

def agegroup(age):                                                                         # возрастная группа
    if age < 18:
        return '0-18'
    elif age >=18 and age <40:
        return '18-40'
    elif age >=40 and age <60:
        return '40-60'
    elif age >=60 and age <80:
        return '60-80'
    else:
        return '80+'

In [12]:
df = pd.read_csv('C:\\Users\\filip\\Downloads\\data_test.csv', parse_dates = ['service_date', 'date_of_birth'])
df = df.query('service_amount != 0').query('service_number!= 0').query('service_amount < 500000') # убираем лишние строки
df['age'] = ((df.service_date - df.date_of_birth) / np.timedelta64 ( 1 , 'Y')).astype('int')  # вычисляем возраст
df['age_gr'] = df.age.apply(agegroup)                                                         # добавляем возрастную группу
df['case'] = (df.service_date - df.sort_values(['insured', 'l_code', 'service_date']).service_date.shift(1))/np.timedelta64 ( 1 , 'D')
                                                        # считаем разницу между датами визитов

In [13]:
df.case = df.case.fillna(15) 
df.sex_id = df.sex_id.fillna('no')
df['sale'] = df.service_amount*df.service_number                                         # считаем сумму продажи
df['month'] = df.service_date.dt.to_period("M")

In [4]:
df1 = df.groupby('l_code', as_index=False).agg({'sale' : 'count'}).sort_values('sale', ascending=False).head(10)
filter_list = df1.l_code.unique()
df_sr_chek = df.query('l_code.isin(@filter_list)').groupby('month', as_index=False).agg({'sale' : 'mean'})
                                                                                         # средний чек по месяцам топ 10 клиник

In [5]:
df2 = df.groupby(['service_date', 'sex_id', 'age_gr', 'insured', 'l_code'], as_index=False)\
       .agg({'record_id' : 'count', 'sale' : 'sum'})\
       .rename(columns={'record_id' : 'visits', 'sale' : 'sale_sum'})

In [14]:
df3 = df.query('case > 14 or case < 0').groupby(['service_date', 'sex_id', 'age_gr', 'insured', 'l_code'], as_index=False)\
       .agg({'case' : 'count'})
df3.case.sum()

444594

In [7]:
df2['month'] = df2.service_date.dt.to_period("M")
df3['month'] = df3.service_date.dt.to_period("M")

In [8]:
df2 = df2.groupby(['month', 'sex_id', 'age_gr', 'insured'], as_index=False)\
       .agg({'visits' : 'count', 'sale_sum' : 'sum'})
df3 = df3.groupby(['month', 'sex_id', 'age_gr', 'insured'], as_index=False)\
       .agg({'case' : 'sum'})
df3.case.sum()

In [9]:
df_vis = df2.groupby(['month', 'sex_id', 'age_gr'], as_index=False)\
       .agg({'visits' : 'sum', 'sale_sum' : 'sum', 'insured' : 'count'})
df_case = df3.groupby(['month', 'sex_id', 'age_gr'], as_index=False)\
       .agg({'case' : 'sum'})

In [10]:
df = 0
df1 = 0
df2 = 0
df3 = 0

In [11]:
df_case.case.sum()

444594